<a href="https://colab.research.google.com/github/joo9906/AI_study/blob/main/coding_challenge(internship)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Coding Challenge: LLM Performance Optimization

### Overview
Your task is to enhance the performance of a small language model (e.g. Qwen2.5-0.5B) on mathematical reasoning tasks. You'll have the freedom to explore various optimization techniques while maintaining reproducibility and providing clear documentation of your methodology.

### Challenge Requirements
* Improve the model's performance on the GSM8K benchmark using Qwen2.5-0.5B as your foundation model
* Document your experimental process and methodology thoroughly
* Ensure your solution is fully reproducible
* Submit all necessary code, model checkpoints, and documentation

### Available Optimization Approaches
You have flexibility in your approach and can explore techniques such as:
* Fine-tuning strategy optimization
* Custom architecture modifications
* Dataset curation and synthesis
* Hyperparameter optimization
* Template and tokenizer configuration adjustments

### Technical Guidelines
* While our baseline implementation uses liger-kernel, you're welcome to explore alternative optimization methods (e.g., PEFT, spectrum)
* You can implement custom components such as:
  * Custom dataset classes
  * Specialized data collators
  * Modified training loops
* You may leverage larger models (>7B) for data synthesis or knowledge distillation

### Evaluation Criteria
* Primary metric: GSM8K benchmark performance
  * Baseline score (Qwen2.5-0.5B-Instruct): 41.6
  * Evaluation using EleutherAI's lm-evaluation-harness

Note: Even if significant score improvements aren't achieved, strong technical analysis and well-reasoned experimentation will be valued highly.

### Deliverables
Required:
* Complete notebook (ipynb or Google Colab format)
* Final model weights and tokenizer (shared via HuggingFace Hub)

Optional:
* Supplementary analysis report (PDF)
* Additional experimental results and ablation studies

In [ ]:
!python -m pip install --upgrade pip -q -U
!pip install -q -U datasets
!pip install -q -U transformers
!pip install -q -U trl
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U fla                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          sh-attn
!pip install -q -U liger-kernel
!pip install -q -U huggingface_hub
!pip install -q -U vllm
!pip install -q -U mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement fla (from versions: none)
ERROR: No matching distribution found for fla
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.0.1 which is incompatible.
ERROR: pip's dependency resolver does not curren

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# (Optional) Mount Google Drive, if you are not using Colab, please comment out the code below.
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [ ]:
#(Optional) 구글 드라이브를 사용할 경우 아래의 코드를 통해 모델을 캐싱하여 시간을 절약하고 학습 데이터를 드라이브에 저장할 수 있습니다.
# If you're running Jupyter notebook in local, set your local caching directory in `cache_dir`.
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import os
cache_dir = "/content/drive/MyDrive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists

In [ ]:
model_id = "Qwen/Qwen2.5-0.5B"

local_path = model_id
local_save_path = os.path.join(cache_dir, local_path)

In [ ]:
from huggingface_hub import snapshot_download
import os

def download_model_repo(repo_id, local_dir):
    # Download the whole repository to the specified local directory
    repo_path = snapshot_download(repo_id=repo_id,
                                  cache_dir=local_dir,
                                  local_dir=local_dir,
                                  local_dir_use_symlinks=False)

    print(f"Repository is saved to: {repo_path}")

def main():
    download_model_repo(model_id, local_save_path)
    print()

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Repository is saved to: /content/drive/MyDrive/huggingface_cache/Qwen/Qwen2.5-0.5B



In [ ]:
from datasets import load_dataset

ds = load_dataset("AI-MO/NuminaMath-CoT", split="train")
ds[0]

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

data/train-00001-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

data/train-00002-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

data/train-00003-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

data/train-00004-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/166k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/859494 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

{'source': 'synthetic_math',
 'problem': 'Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$.',
 'solution': 'For an arithmetic sequence, the difference between consecutive terms must be equal. Therefore, we can set up the following equations based on the sequence given:\n\\[ (y + 2) - \\left(-\\frac{1}{3}\\right) = 4y - (y+2) \\]\n\nSimplify and solve these equations:\n\\[ y + 2 + \\frac{1}{3} = 4y - y - 2 \\]\n\\[ y + \\frac{7}{3} = 3y - 2 \\]\n\\[ \\frac{7}{3} + 2 = 3y - y \\]\n\\[ \\frac{13}{3} = 2y \\]\n\\[ y = \\frac{13}{6} \\]\n\nThus, the value of $y$ that satisfies the given arithmetic sequence is $\\boxed{\\frac{13}{6}}$.',
 'messages': [{'content': 'Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$.',
   'role': 'user'},
  {'content': 'For an arithmetic sequence, the difference between consecutive terms must be equal. Therefore, we can set up the following equations based on the sequence

In [ ]:
# randomly sample 20000 examples
sampled_ds = ds.shuffle(seed=42).select(range(20000))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(
    local_save_path,
    device_map='auto',
    dtype=torch.float16,
    cache_dir=cache_dir)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          padding_side='left',
                                          truncation_side='left')

`torch_dtype` is deprecated! Use `dtype` instead!


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
chat_template = "{%- if tools %}\n    {{- '<|im_start|>system\\n' }}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- messages[0]['content'] }}\n    {%- else %}\n        {{- 'You are Alli, created by Allganize. You are a helpful assistant.' }}\n    {%- endif %}\n    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}\n    {%- for tool in tools %}\n        {{- \"\\n\" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}\n    {%- else %}\n        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}\n    {%- endif %}\n{%- endif %}\n{%- for message in messages %}\n    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) or (message.role == \"assistant\" and not message.tool_calls) %}\n        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}\n    {%- elif message.role == \"assistant\" %}\n        {{- '<|im_start|>' + message.role }}\n        {%- if message.content %}\n            {{- '\\n' + message.content }}\n        {%- endif %}\n        {%- for tool_call in message.tool_calls %}\n            {%- if tool_call.function is defined %}\n                {%- set tool_call = tool_call.function %}\n            {%- endif %}\n            {{- '\\n<tool_call>\\n{\"name\": \"' }}\n            {{- tool_call.name }}\n            {{- '\", \"arguments\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- '}\\n</tool_call>' }}\n        {%- endfor %}\n        {{- '<|im_end|>\\n' }}\n    {%- elif message.role == \"tool\" %}\n        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}\n            {{- '<|im_start|>user' }}\n        {%- endif %}\n        {{- '\\n<tool_response>\\n' }}\n        {{- message.content }}\n        {{- '\\n</tool_response>' }}\n        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}\n            {{- '<|im_end|>\\n' }}\n        {%- endif %}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\\n' }}\n{%- endif %}\n"

tokenizer.chat_template = chat_template
tokenizer.eos_token = "<|im_end|>"
tokenizer.eos_token_id = 151645

model.config.eos_token_id = tokenizer.eos_token_id
model.generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
def format_prompt_func(sample):
  sample['text'] = tokenizer.apply_chat_template(sample['messages'], tokenize = False, add_generation_prompt = False)
  return sample['text']

In [ ]:
# sampled_ds = sampled_ds.map(format_prompt_func, num_proc=os.cpu_count())
sampled_ds = sampled_ds.train_test_split(test_size=0.1, seed=42)

In [ ]:
import trl
print(trl.__version__)

0.24.0


In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer, SFTConfig
import mlflow
import os

# mlflow 초기 세팅 및 이전 기록이 돌아가고 있을 수 있으니 end run
mlflow.set_tracking_uri("file:///content/mlruns")
mlflow.set_experiment("Allganize_optimize_qwen2.5B")
mlflow.end_run()

with mlflow.start_run(run_name="Colab_T4_Qwen_finetune"):
  mlflow.log_param("model_id", "Qwen/Qwen2.5-0.5B")
  mlflow.log_param("dataset", "AI-MO/NuminaMath-CoT")

  training_arguments = SFTConfig(
      dataset_text_field='text',
      output_dir=os.path.join(cache_dir, "results"),
      num_train_epochs=1,
      per_device_train_batch_size=1,
      per_device_eval_batch_size=1,
      gradient_accumulation_steps=4,
      optim="paged_adamw_8bit",
      save_strategy='epoch',
      eval_steps=0.1,
      logging_steps=10,
      learning_rate=5e-6,
      weight_decay=0.01,
      # max_seq_len=2048,   # SFTConfig 0.24.0에선 사라짐.
      max_grad_norm=1,
      max_steps=-1,
      warmup_ratio=0.05,
      packing=True,
      lr_scheduler_type="cosine",
      #use_liger=True,    # SFTConfig 0.24.0에선 사라짐.
  )

  trainer = SFTTrainer(
      model=model,
      train_dataset=sampled_ds['train'],
      eval_dataset=sampled_ds['test'],
      formatting_func=format_prompt_func,
      args=training_arguments,
  )

  result = trainer.train()

  mlflow.log_metric("train_loss", result.training_loss)
  mlflow.log_artifact(os.path.join(cache_dir, "results"))

mlflow.end_run()

Applying formatting function to train dataset:   0%|          | 0/18000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/18000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/18000 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: happyboyjoo (happyboyjoo-noldolmal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.993200
20,1.030300
30,0.974800
40,0.831500
50,0.792700
60,0.871100
70,0.794600
80,0.680900
90,0.747000
100,0.746100


In [ ]:
import os

new_model_name = "allganize/qwen0.5b-tech-jooyoung" # please specify your own repo/model id
output_dir = os.path.join(cache_dir, new_model_name)

model.config.use_cache = True
model.save_pretrained(output_dir, safe_serialization=True)
tokenizer.save_pretrained(output_dir)

NameError: name 'cache_dir' is not defined

In [ ]:
model.push_to_hub(repo_id=new_model_name, token=True, max_shard_size="5GB", safe_serialization=True)
tokenizer.push_to_hub(repo_id=new_model_name, token=True)

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
!cd lm-evaluation-harness && pip install -e .

In [ ]:
eval_output_path = os.path.join(cache_dir, "results", "gsm8k")
os.makedirs(eval_output_path, exist_ok=True)

# It takes about 11 minutes on a single A100 40GB GPU (about 100 minutes on a single T4 GPU)
eval_output_path = os.path.join(eval_output_path, "result-original.json")
tasks = "gsm8k"

# eval_cmd = f"""
# lm_eval --model vllm \
#     --model_args pretrained={new_model_name},trust_remote_code=True,dtype=float16 \
#     --tasks {tasks} \
#     --device cuda:0 \
#     --batch_size auto:4 \
#     --output_path {eval_output_path}
# """

eval_cmd = f"""
lm_eval --model vllm \
    --model_args pretrained={new_model_name},trust_remote_code=True,dtype=float16 \
    --tasks {tasks} \
    --device cuda:0 \
    --batch_size auto:4 \
    --output_path {eval_output_path}
"""

In [ ]:
# run an evaluation command
!{eval_cmd}

In [ ]:
# Qwen2.5-0.5B-Insruct results for the reference
#vllm (pretrained=Qwen/Qwen2.5-0.5B-Instruct,trust_remote_code=True,dtype=float16), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: auto:4
#|Tasks|Version|     Filter     |n-shot|  Metric   |   |Value |   |Stderr|
#|-----|------:|----------------|-----:|-----------|---|-----:|---|-----:|
#|gsm8k|      3|flexible-extract|     5|exact_match|↑  |0.3442|±  |0.0131|
#|     |       |strict-match    |     5|exact_match|↑  |0.3169|±  |0.0128|